In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from google.colab import files
import io

In [12]:
uploaded = files.upload()

Saving movies.csv to movies (1).csv
Saving ratings.csv to ratings (1).csv


In [13]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [14]:
data = pd.merge(ratings, movies, on='movieId')

In [15]:
def filter_movies_by_genre(data, genre):
    return data[data['genres'].str.contains(genre, case=False, na=False)]


In [16]:
def create_user_item_matrix(data):
    user_item_matrix = data.pivot_table(index='userId', columns='title', values='rating')
    user_item_matrix.fillna(0, inplace=True)
    return user_item_matrix

In [17]:
# Function to build and train the autoencoder model
def build_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(128, activation='relu')(input_layer)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')
    return autoencoder

In [21]:
from sklearn.metrics import mean_squared_error

In [22]:
 #Function to make movie recommendations within a genre
def recommend_movies(user_id, genre, n_recommendations):
    filtered_data = filter_movies_by_genre(data, genre)
    user_item_matrix = create_user_item_matrix(filtered_data)

    # Normalize the user-item matrix
    user_item_matrix_norm = user_item_matrix / user_item_matrix.max().max()

    # Split the data into training and test sets
    train_data, test_data = train_test_split(user_item_matrix_norm, test_size=0.2, random_state=42)

    # Build and train the autoencoder
    input_dim = user_item_matrix.shape[1]
    autoencoder = build_autoencoder(input_dim)
    history = autoencoder.fit(train_data, train_data, epochs=50, batch_size=256, shuffle=True, validation_data=(test_data, test_data), verbose=1)

    # Print the model performance
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    print("Final training loss:", train_loss[-1])
    print("Final validation loss:", val_loss[-1])

    # Calculate and print RMSE for training and validation sets
    train_predictions = autoencoder.predict(train_data)
    val_predictions = autoencoder.predict(test_data)
    train_rmse = np.sqrt(mean_squared_error(train_data.values.flatten(), train_predictions.flatten()))
    val_rmse = np.sqrt(mean_squared_error(test_data.values.flatten(), val_predictions.flatten()))
    print("Training RMSE:", train_rmse)
    print("Validation RMSE:", val_rmse)

    # Get the user's ratings
    user_ratings = user_item_matrix_norm.loc[user_id].values.reshape(1, -1)

    # Predict the ratings using the autoencoder
    predicted_ratings = autoencoder.predict(user_ratings)

    # Get the top N recommendations
    recommendations = []
    for i in np.argsort(predicted_ratings[0])[::-1][:n_recommendations]:
        recommendations.append(user_item_matrix.columns[i])

    return recommendations

In [23]:
preferred_genre = input("Enter your preferred genre: ")
user_id = 1  # For example, user ID 1
recommended_movies = recommend_movies(user_id, preferred_genre, 5)
print("Recommended movies:", recommended_movies)

Enter your preferred genre: War


Epoch 1/50
2/2 [==============================] - 1s 229ms/step - loss: 0.2467 - val_loss: 0.2447
Epoch 2/50
2/2 [==============================] - 0s 37ms/step - loss: 0.2454 - val_loss: 0.2427
Epoch 3/50
2/2 [==============================] - 0s 37ms/step - loss: 0.2433 - val_loss: 0.2395
Epoch 4/50
2/2 [==============================] - 0s 38ms/step - loss: 0.2401 - val_loss: 0.2347
Epoch 5/50
2/2 [==============================] - 0s 37ms/step - loss: 0.2351 - val_loss: 0.2274
Epoch 6/50
2/2 [==============================] - 0s 40ms/step - loss: 0.2278 - val_loss: 0.2168
Epoch 7/50
2/2 [==============================] - 0s 51ms/step - loss: 0.2172 - val_loss: 0.2021
Epoch 8/50
2/2 [==============================] - 0s 40ms/step - loss: 0.2026 - val_loss: 0.1827
Epoch 9/50
2/2 [==============================] - 0s 38ms/step - loss: 0.1829 - val_loss: 0.1587
Epoch 10/50
2/2 [==============================] - 0s 41ms/step - loss: 0.1586 - val_loss: 0.1309
Epoch 11/50
2/2 [===========